In [1]:
import pydicom
import numpy as np
import os

def load_dicom_series(directory):
    slices = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        ds = pydicom.dcmread(filepath)
        slices.append(ds)
    
    # 按照切片的位置（Image Position Patient）排序
    slices.sort(key=lambda x: x.ImagePositionPatient[2])
    
    return slices

# 示例：加载 DICOM 系列
directory = r"dicomPath"
slices = load_dicom_series(directory)


In [2]:
# 提取一个切片的关键元数据
ds = slices[0]
IPP = np.array(ds.ImagePositionPatient)
IOP = np.array(ds.ImageOrientationPatient)
PS = np.array(ds.PixelSpacing)
slice_thickness = ds.SliceThickness


In [3]:
# 将 IOP 拆分为行方向和列方向向量
row_vector = IOP[:3]
col_vector = IOP[3:]

# 计算每个像素的物理坐标
def compute_patient_coordinates(IPP, row_vector, col_vector, PS, i, j):
    x = IPP[0] + i * row_vector[0] * PS[0] + j * col_vector[0] * PS[1]
    y = IPP[1] + i * row_vector[1] * PS[0] + j * col_vector[1] * PS[1]
    z = IPP[2] + i * row_vector[2] * PS[0] + j * col_vector[2] * PS[1]
    return np.array([x, y, z])

# 示例：计算像素 (i, j) = (100, 150) 的患者坐标
i, j = 100, 150
patient_coord = compute_patient_coordinates(IPP, row_vector, col_vector, PS, i, j)
print(f"Pixel (i, j) = ({i}, {j}) in patient coordinate system: {patient_coord}")


Pixel (i, j) = (100, 150) in patient coordinate system: [ -21.50418055   75.48817786 -609.39688217]


In [4]:
def convert_dicom_series_to_patient_coordinates(slices):
    volume = []
    for ds in slices:
        IPP = np.array(ds.ImagePositionPatient)
        IOP = np.array(ds.ImageOrientationPatient)
        PS = np.array(ds.PixelSpacing)
        row_vector = IOP[:3]
        col_vector = IOP[3:]
        
        rows, cols = ds.pixel_array.shape
        slice_coords = np.zeros((rows, cols, 3))
        
        for i in range(rows):
            for j in range(cols):
                slice_coords[i, j] = compute_patient_coordinates(IPP, row_vector, col_vector, PS, i, j)
        
        volume.append(slice_coords)
    
    return np.array(volume)

# 转换整个 DICOM 系列为三维坐标
patient_coordinates_volume = convert_dicom_series_to_patient_coordinates(slices)


In [5]:
patient_coordinates_volume.max(axis=(0, 1, 2))

array([  93.84068064,  158.50361412, -395.68208832])

In [6]:
def determine_resolution(slice):
    ps = slice.PixelSpacing  # Pixel Spacing, usually a list [row_spacing, col_spacing]
    st = slice.SliceThickness  # Slice Thickness
    ss = getattr(slice, 'SpacingBetweenSlices', st)  # Slice Spacing, if exists

    # 分辨率即为每个体素在患者坐标系中的物理尺寸
    resolution = np.array([ps[1], ps[0], ss])  # [x_spacing, y_spacing, z_spacing]
    
    return resolution

# 示例：从一个 DICOM 切片中提取分辨率
resolution = determine_resolution(slices[0])
print("Resolution (mm):", resolution)

Resolution (mm): [0.5 0.5 3.5]


### 3. 根据所有切片的一致性验证分辨率
在某些情况下，DICOM 文件中的 Pixel Spacing 和 Slice Thickness 可能会略有不同（通常不常见），这可能会导致分辨率在不同切片间存在差异。因此，建议在处理所有切片之前，检查这些值是否一致。

In [7]:
def check_resolution_consistency(slices):
    first_resolution = determine_resolution(slices[0])
    
    for slice in slices[1:]:
        resolution = determine_resolution(slice)
        if not np.allclose(first_resolution, resolution, rtol=1e-5):
            print("Warning: Inconsistent resolution detected across slices.")
            break
    
    return first_resolution

# 检查所有切片的分辨率一致性
resolution = check_resolution_consistency(slices)


### 处理切片分辨率不一致的步骤
1. 检查不一致性
首先，检查所有切片的分辨率，以确定是否存在显著差异。这包括检查 Pixel Spacing（即 x 和 y 方向上的分辨率）以及 Slice Thickness 或 Spacing Between Slices（z 方向上的分辨率）。

In [8]:
def check_resolution_consistency(slices):
    first_resolution = determine_resolution(slices[0])
    resolutions = []
    
    for slice in slices:
        resolution = determine_resolution(slice)
        resolutions.append(resolution)
        if not np.allclose(first_resolution, resolution, rtol=1e-5):
            print(f"Resolution inconsistency found in slice with SOPInstanceUID {slice.SOPInstanceUID}: {resolution}")
    
    return resolutions

# 检查所有切片的分辨率
resolutions = check_resolution_consistency(slices)


2. 选择统一的分辨率
   
    在处理不一致的分辨率时，可以选择一个统一的分辨率。通常有两种方法：
    + 最大公约数法：选择一个所有切片都能整除的最小公约分辨率。
    + 平均分辨率法：计算所有切片分辨率的平均值，并将其作为统一分辨率

In [9]:
# 计算平均分辨率
mean_resolution = np.mean(resolutions, axis=0)
print("Mean Resolution:", mean_resolution)

Mean Resolution: [0.5 0.5 3.5]


3. 重采样（Resampling）切片

    将每个切片重采样到统一的分辨率。这可以使用插值技术将不同分辨率的切片调整为相同的目标分辨率。可以使用 scipy.ndimage 中的 zoom 函数来实现这一点。

In [ ]:
from scipy.ndimage import zoom

def resample_slice(slice, target_resolution):
    original_resolution = determine_resolution(slice)
    zoom_factors = original_resolution / target_resolution
    
    # 对像素数据进行重采样
    resampled_data = zoom(slice.pixel_array, zoom_factors, order=1)  # 使用线性插值 (order=1)
    
    # 更新切片数据
    slice.pixel_array = resampled_data
    slice.PixelSpacing = [target_resolution[1], target_resolution[0]]
    if hasattr(slice, 'SliceThickness'):
        slice.SliceThickness = target_resolution[2]
    
    return slice

# 对所有切片进行重采样
target_resolution = mean_resolution  # 或者选择最大公约数
resampled_slices = [resample_slice(slice, target_resolution) for slice in slices]


In [ ]:
import numpy as np

def initialize_volume(min_coord, max_coord, resolution):
    # 计算三维体积的尺寸
    volume_shape = np.ceil((max_coord - min_coord) / resolution).astype(int)
    # 初始化三维体积，通常用零填充
    volume = np.zeros(volume_shape)
    return volume, volume_shape

# 假设 min_coord 和 max_coord 是已知的三维空间的边界
# 假设 resolution 是一个三维向量，代表 x, y, z 方向上的分辨率
volume, volume_shape = initialize_volume(patient_coordinates_volume.min(axis=(0, 1, 2)), patient_coordinates_volume.max(axis=(0, 1, 2)), resolution)
